In [3]:
import os
import json
import supervisely as sly
from model_benchmark import sly2coco

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
gt_dataset_id = 87510
dt_dataset_id = 87612
save_path = 'data/model-benchmark'

In [47]:
api = sly.Api()

gt_dataset_info = api.dataset.get_info_by_id(gt_dataset_id)
gt_dataset_name = gt_dataset_info.name
gt_project_id = gt_dataset_info.project_id
gt_project_info = api.project.get_info_by_id(gt_project_id)
gt_project_name = gt_project_info.name

dt_dataset_info = api.dataset.get_info_by_id(dt_dataset_id)
dt_dataset_name = dt_dataset_info.name
dt_project_id = dt_dataset_info.project_id
dt_project_info = api.project.get_info_by_id(dt_project_id)
dt_project_name = dt_project_info.name

assert gt_dataset_name == dt_dataset_name

save_project_path = os.path.join(save_path, dt_project_name)
gt_project_path = os.path.join(save_project_path, "gt_dataset")
dt_project_path = os.path.join(save_project_path, "dt_dataset")

In [48]:
# Download projects
sly.download(api, gt_project_id, gt_project_path, dataset_ids=[gt_dataset_id], log_progress=True, save_images=False)
# sly.download(api, dt_project_id, dt_project_path, dataset_ids=[dt_dataset_id], log_progress=True, save_images=False)

{"message": "progress", "event_type": "EventType.PROGRESS", "subtask": "Downloading dataset: 'val2017'", "current": 0, "total": 5000, "timestamp": "2024-05-22T11:22:46.246Z", "level": "info"}
{"message": "progress", "event_type": "EventType.PROGRESS", "subtask": "Downloading dataset: 'val2017'", "current": 50, "total": 5000, "timestamp": "2024-05-22T11:22:47.347Z", "level": "info"}
{"message": "progress", "event_type": "EventType.PROGRESS", "subtask": "Downloading dataset: 'val2017'", "current": 100, "total": 5000, "timestamp": "2024-05-22T11:22:48.251Z", "level": "info"}
{"message": "progress", "event_type": "EventType.PROGRESS", "subtask": "Downloading dataset: 'val2017'", "current": 150, "total": 5000, "timestamp": "2024-05-22T11:22:49.033Z", "level": "info"}
{"message": "progress", "event_type": "EventType.PROGRESS", "subtask": "Downloading dataset: 'val2017'", "current": 200, "total": 5000, "timestamp": "2024-05-22T11:22:49.945Z", "level": "info"}
{"message": "progress", "event_ty

In [53]:
from model_benchmark import sly2coco
from importlib import reload
reload(sly2coco)

<module 'model_benchmark.sly2coco' from '/root/model-benchmark/model_benchmark/sly2coco.py'>

In [ ]:
# %%time
# cocoGt = sly2coco.sly2coco(gt_project_path, gt_dataset_name, is_dt_dataset=False)
# cocoDt = sly2coco.sly2coco(dt_project_path, dt_dataset_name, is_dt_dataset=True)

# assert cocoDt['categories'] == cocoGt['categories']
# assert all([img1['file_name'] == img1['file_name'] for img1, img2 in zip(cocoGt['images'], cocoDt['images'])])

In [54]:
cocoGt = sly2coco.sly2coco(gt_project_path, gt_dataset_name, is_dt_dataset=False, accepted_shapes=['rectangle'])
cocoGt_path = os.path.join(save_project_path, "cocoGt.json")
with open(cocoGt_path, 'w') as f:
    json.dump(cocoGt, f)

In [37]:
cocoDt = sly2coco.sly2coco(dt_project_path, dt_dataset_name, is_dt_dataset=True)
cocoDt_path = os.path.join(save_project_path, "cocoDt.json")
with open(cocoDt_path, 'w') as f:
    json.dump(cocoDt['annotations'], f)

In [44]:
# Remap cocoGt ids to match cocoDt ids
with open("cocoGt.json", 'r') as f:
    cocoGt = json.load(f)

img_id_map = {}
images_sorted = sorted(cocoGt["images"], key=lambda x: x["file_name"])
for i, img in enumerate(images_sorted):
    img_id_map[img["id"]] = i
    img["id"] = i
cocoGt["images"] = images_sorted

# Remap annotations (catIds)
cat_id_map = {}
for cat_id_dt in cocoDt["categories"]:
    for cat_id_gt in cocoGt["categories"]:
        if cat_id_dt["name"] == cat_id_gt["name"]:
            cat_id_map[cat_id_gt["id"]] = cat_id_dt["id"]
            break
for ann in cocoGt["annotations"]:
    ann["category_id"] = cat_id_map[ann["category_id"]]
    ann["image_id"] = img_id_map[ann["image_id"]]
cocoGt["categories"] = cocoDt["categories"]

with open("cocoGt_sorted.json", 'w') as f:
    json.dump(cocoGt, f)

In [ ]:
# rtdetr-50 ep200: 0.177
# rtdetr-50 ep60: 0.213
# rtdetr-50 EMA ep60: 0.189
# rtdetr-50 MODEL ep60: 0.188